<a href="https://colab.research.google.com/github/ThalesCavalcanti/MachineLearningGroundwork/blob/main/Parte_3_1_Regras_de_associac%CC%A7a%CC%83o_Algoritmo_apriori_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

Reconecte ao Drive.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
DRIVE_DIRECTORY = "curso_ml"
DRIVE_DIRECTORY = os.path.join("/content/drive/MyDrive/cursoML", DRIVE_DIRECTORY)



---



In [4]:
!pip install apyori

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5954 sha256=415b35863f5624d1abe160126e1abcc5291a8eaaf7a0730fa6795d33c469bced
  Stored in directory: /root/.cache/pip/wheels/77/3d/a6/d317a6fb32be58a602b1e8c6b5d6f31f79322da554cad2a5ea
Successfully built apyori


In [5]:
import kagglehub
import os
import pandas as pd

from apyori import apriori

# Base de dados Market Basket Analysis

## Download e preparação dos dados

Desta vez, você vai utilizar uma base de dados disponível no Kaggle, que contém um histórico de aquisição de itens em uma loja. Cada linha representa a transação de um item, com o ID da compra, quantidade, data, preço, ID do cliente e país.

https://www.kaggle.com/datasets/aslanahmedov/market-basket-analysis

> * Número de Linhas: 522065
> * Número de Atributos: 7
>   * BillNo: Número de 6 dígitos atribuído a cada transação. Nominal.
>   * Itemname: Nome do produto. Nominal.
>   * Quantity: As quantidades de cada produto por transação. Numérico.
>   * Date: O dia e a hora em que cada transação foi gerada. Numérico.
>   * Price: Preço do produto. Numérico.
>   * CustomerID: Número de 5 dígitos atribuído a cada cliente. Nominal.
>   * Country: Nome do país onde cada cliente reside. Nominal.

In [6]:
path = kagglehub.dataset_download("aslanahmedov/market-basket-analysis")

print("Path to dataset files:", path)

100%|██████████| 22.8M/22.8M [00:01<00:00, 15.0MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/aslanahmedov/market-basket-analysis/versions/2


O dataset foi baixado para a pasta indica, vamos inspecionar o conteúdo desta pasta.

In [7]:
!ls {path}

Assignment-1_Data.csv  Assignment-1_Data.xlsx


O dataset é fornecido em formato `csv` e `xlsx` (Excel). Nós vamos utilizar o arquivo `csv`, então vamos inspecionar suas primeiras linhas.

In [8]:
!head {path}/Assignment-1_Data.csv

﻿BillNo;Itemname;Quantity;Date;Price;CustomerID;Country
536365;WHITE HANGING HEART T-LIGHT HOLDER;6;01.12.2010 08:26;2,55;17850;United Kingdom
536365;WHITE METAL LANTERN;6;01.12.2010 08:26;3,39;17850;United Kingdom
536365;CREAM CUPID HEARTS COAT HANGER;8;01.12.2010 08:26;2,75;17850;United Kingdom
536365;KNITTED UNION FLAG HOT WATER BOTTLE;6;01.12.2010 08:26;3,39;17850;United Kingdom
536365;RED WOOLLY HOTTIE WHITE HEART.;6;01.12.2010 08:26;3,39;17850;United Kingdom
536365;SET 7 BABUSHKA NESTING BOXES;2;01.12.2010 08:26;7,65;17850;United Kingdom
536365;GLASS STAR FROSTED T-LIGHT HOLDER;6;01.12.2010 08:26;4,25;17850;United Kingdom
536366;HAND WARMER UNION JACK;6;01.12.2010 08:28;1,85;17850;United Kingdom
536366;HAND WARMER RED POLKA DOT;6;01.12.2010 08:28;1,85;17850;United Kingdom


O arquivo contém um cabeçalho, e os campos são separados por `;`. Então, vamos carregar no Python levando isso em consideração.

In [9]:
mba = pd.read_csv(os.path.join(path, "Assignment-1_Data.csv"), delimiter=";")
mba

/tmp/ipython-input-2657018351.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  mba = pd.read_csv(os.path.join(path, "Assignment-1_Data.csv"), delimiter=";")


,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
...,...,...,...,...,...,...,...
522059,581587,PACK OF 20 SPACEBOY NAPKINS,12,09.12.2011 12:50,"0,85",12680.0,France
522060,581587,CHILDREN'S APRON DOLLY GIRL,6,09.12.2011 12:50,"2,1",12680.0,France
522061,581587,CHILDRENS CUTLERY DOLLY GIRL,4,09.12.2011 12:50,"4,15",12680.0,France
522062,581587,CHILDRENS CUTLERY CIRCUS PARADE,4,09.12.2011 12:50,"4,15",12680.0,France


Perceba que são mais de 500 mil transações.

Vamos contar quantas vezes cada item aparece neste dataset.

In [10]:
item_freq = mba.groupby("Itemname")["Itemname"].count().sort_values(ascending=False)
item_freq

,Itemname
Itemname,
WHITE HANGING HEART T-LIGHT HOLDER,2269
JUMBO BAG RED RETROSPOT,2087
REGENCY CAKESTAND 3 TIER,1930
PARTY BUNTING,1677
LUNCH BAG RED RETROSPOT,1570
...,...
wrongly marked. 23343 in box,1
wrongly marked carton 22804,1
wet boxes,1


Alguns itens são bem comuns, mas outros foram comprados uma única vez. Vamos fazer uma limpeza nesta dataset, caso contrário o tempo de execução do algoritmo será muito longo.

Vamos manter apenas os itens que aparecem pelo menos 1000 vezes.

In [11]:
item_filt = item_freq[item_freq >= 1000].index.tolist()
len(item_filt)

34

A próxima célula mantém apenas as transações que contém os 34 itens selecionados.

In [12]:
mba_filt = mba[mba["Itemname"].isin(item_filt)]
mba_filt

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
9,536367,ASSORTED COLOUR BIRD ORNAMENT,32,01.12.2010 08:34,"1,69",13047.0,United Kingdom
21,536368,JAM MAKING SET WITH JARS,6,01.12.2010 08:34,"4,25",13047.0,United Kingdom
27,536370,ALARM CLOCK BAKELIKE RED,24,01.12.2010 08:45,"3,75",12583.0,France
45,536370,POSTAGE,3,01.12.2010 08:45,18,12583.0,France
...,...,...,...,...,...,...,...
522026,581585,VICTORIAN GLASS HANGING T-LIGHT,12,09.12.2011 12:31,"1,95",15804.0,United Kingdom
522029,581585,RABBIT NIGHT LIGHT,12,09.12.2011 12:31,"2,08",15804.0,United Kingdom
522030,581585,ASSORTED COLOUR BIRD ORNAMENT,16,09.12.2011 12:31,"1,69",15804.0,United Kingdom
522035,581585,ALARM CLOCK BAKELIKE RED,4,09.12.2011 12:31,"3,75",15804.0,United Kingdom


Agora vamos ver quantas compras aparecem, agregando pelo atributo `BillNo`. Neste caso, já estamos também selecionando os itens de cada transação, e formatando como lista.

In [13]:
baskets = mba_filt.groupby("BillNo")["Itemname"].agg(list).tolist()
len(baskets)

12788

Vamos inspecionar as duas primeiras compras.

In [14]:
baskets[:2]

[['WHITE HANGING HEART T-LIGHT HOLDER'], ['ASSORTED COLOUR BIRD ORNAMENT']]

Veja que elas correspondem às primeiras duas linhas do DataFrame acima.

Por fim, precisamos ordenar esta lista em ordem decrescente de itens, para permitir sua importação com o Pandas mais tarde.

In [15]:
baskets = sorted(baskets, key=len, reverse=True)

Vamos inspecionar de novo as duas primeiras compras.

In [16]:
baskets[:2]

[['RED RETROSPOT CHARLOTTE BAG',
  'LUNCH BAG RED RETROSPOT',
  'LUNCH BAG WOODLAND',
  'LUNCH BAG  BLACK SKULL.',
  'LUNCH BAG CARS BLUE',
  'PACK OF 72 RETROSPOT CAKE CASES',
  'JUMBO STORAGE BAG SUKI',
  "PAPER CHAIN KIT 50'S CHRISTMAS",
  'VICTORIAN GLASS HANGING T-LIGHT',
  'LUNCH BAG SPACEBOY DESIGN',
  'LUNCH BAG SUKI DESIGN',
  'LUNCH BAG PINK POLKADOT',
  'JUMBO BAG PINK POLKADOT',
  'JUMBO SHOPPER VINTAGE RED PAISLEY',
  'REGENCY CAKESTAND 3 TIER',
  'NATURAL SLATE HEART CHALKBOARD',
  'HEART OF WICKER SMALL',
  'RECIPE BOX PANTRY YELLOW DESIGN',
  'GREEN REGENCY TEACUP AND SAUCER',
  'ROSES REGENCY TEACUP AND SAUCER',
  'SET OF 3 CAKE TINS PANTRY DESIGN',
  'ALARM CLOCK BAKELIKE RED',
  'JAM MAKING SET WITH JARS',
  'JAM MAKING SET PRINTED',
  'SET OF 4 PANTRY JELLY MOULDS',
  'RABBIT NIGHT LIGHT',
  'LUNCH BAG APPLE DESIGN',
  'SPOTTY BUNTING',
  'PARTY BUNTING',
  'WOODEN PICTURE FRAME WHITE FINISH',
  'JUMBO BAG RED RETROSPOT',
  'WHITE HANGING HEART T-LIGHT HOLDER'],
 ['

E as duas últimas.

In [17]:
baskets[-2:]

[['POSTAGE'], ['JUMBO BAG PINK POLKADOT']]

Agora podemos salvar essas compras em um arquivo txt na pasta do Drive. Para evitar confusão com o delimitador, já que um item pode conter vírgulas em sua descrição, vamos manter o caractere `;`.

In [18]:
with open(os.path.join(DRIVE_DIRECTORY, "itemlist.txt"), "w") as f:
    for basket in baskets:
        f.write(";".join(basket) + "\n")

Observe como os arquivos foram salvos.

In [19]:
!head {DRIVE_DIRECTORY}/itemlist.txt

RED RETROSPOT CHARLOTTE BAG;LUNCH BAG RED RETROSPOT;LUNCH BAG WOODLAND;LUNCH BAG  BLACK SKULL.;LUNCH BAG CARS BLUE;PACK OF 72 RETROSPOT CAKE CASES;JUMBO STORAGE BAG SUKI;PAPER CHAIN KIT 50'S CHRISTMAS;VICTORIAN GLASS HANGING T-LIGHT;LUNCH BAG SPACEBOY DESIGN;LUNCH BAG SUKI DESIGN;LUNCH BAG PINK POLKADOT;JUMBO BAG PINK POLKADOT;JUMBO SHOPPER VINTAGE RED PAISLEY;REGENCY CAKESTAND 3 TIER;NATURAL SLATE HEART CHALKBOARD;HEART OF WICKER SMALL;RECIPE BOX PANTRY YELLOW DESIGN;GREEN REGENCY TEACUP AND SAUCER;ROSES REGENCY TEACUP AND SAUCER;SET OF 3 CAKE TINS PANTRY DESIGN;ALARM CLOCK BAKELIKE RED;JAM MAKING SET WITH JARS;JAM MAKING SET PRINTED;SET OF 4 PANTRY JELLY MOULDS;RABBIT NIGHT LIGHT;LUNCH BAG APPLE DESIGN;SPOTTY BUNTING;PARTY BUNTING;WOODEN PICTURE FRAME WHITE FINISH;JUMBO BAG RED RETROSPOT;WHITE HANGING HEART T-LIGHT HOLDER
PACK OF 72 RETROSPOT CAKE CASES;JUMBO STORAGE BAG SUKI;PAPER CHAIN KIT 50'S CHRISTMAS;VICTORIAN GLASS HANGING T-LIGHT;LUNCH BAG SUKI DESIGN;LUNCH BAG PINK POLKADOT;

## Algoritmo apriori

Carregue a lista de compras como um DataFrame do Pandas, lembre-se de selecionar o delimitador adequado.

In [29]:
base = pd.read_csv(os.path.join(DRIVE_DIRECTORY, "itemlist.txt"), delimiter=";", header=None)
base

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,RED RETROSPOT CHARLOTTE BAG,LUNCH BAG RED RETROSPOT,LUNCH BAG WOODLAND,LUNCH BAG BLACK SKULL.,LUNCH BAG CARS BLUE,PACK OF 72 RETROSPOT CAKE CASES,JUMBO STORAGE BAG SUKI,PAPER CHAIN KIT 50'S CHRISTMAS,VICTORIAN GLASS HANGING T-LIGHT,LUNCH BAG SPACEBOY DESIGN,...,JAM MAKING SET WITH JARS,JAM MAKING SET PRINTED,SET OF 4 PANTRY JELLY MOULDS,RABBIT NIGHT LIGHT,LUNCH BAG APPLE DESIGN,SPOTTY BUNTING,PARTY BUNTING,WOODEN PICTURE FRAME WHITE FINISH,JUMBO BAG RED RETROSPOT,WHITE HANGING HEART T-LIGHT HOLDER
1,PACK OF 72 RETROSPOT CAKE CASES,JUMBO STORAGE BAG SUKI,PAPER CHAIN KIT 50'S CHRISTMAS,VICTORIAN GLASS HANGING T-LIGHT,LUNCH BAG SUKI DESIGN,LUNCH BAG PINK POLKADOT,JUMBO SHOPPER VINTAGE RED PAISLEY,REGENCY CAKESTAND 3 TIER,NATURAL SLATE HEART CHALKBOARD,HEART OF WICKER SMALL,...,WHITE HANGING HEART T-LIGHT HOLDER,RED RETROSPOT CHARLOTTE BAG,LUNCH BAG RED RETROSPOT,LUNCH BAG WOODLAND,LUNCH BAG BLACK SKULL.,NaN,NaN,NaN,NaN,NaN
2,PACK OF 72 RETROSPOT CAKE CASES,JUMBO STORAGE BAG SUKI,PAPER CHAIN KIT 50'S CHRISTMAS,VICTORIAN GLASS HANGING T-LIGHT,LUNCH BAG SPACEBOY DESIGN,LUNCH BAG SUKI DESIGN,LUNCH BAG PINK POLKADOT,REGENCY CAKESTAND 3 TIER,NATURAL SLATE HEART CHALKBOARD,RECIPE BOX PANTRY YELLOW DESIGN,...,WHITE HANGING HEART T-LIGHT HOLDER,LUNCH BAG RED RETROSPOT,LUNCH BAG WOODLAND,LUNCH BAG BLACK SKULL.,LUNCH BAG CARS BLUE,NaN,NaN,NaN,NaN,NaN
3,ALARM CLOCK BAKELIKE RED,JAM MAKING SET WITH JARS,JAM MAKING SET PRINTED,SET OF 4 PANTRY JELLY MOULDS,RABBIT NIGHT LIGHT,LUNCH BAG APPLE DESIGN,JUMBO BAG RED RETROSPOT,WHITE HANGING HEART T-LIGHT HOLDER,RED RETROSPOT CHARLOTTE BAG,LUNCH BAG RED RETROSPOT,...,GREEN REGENCY TEACUP AND SAUCER,ROSES REGENCY TEACUP AND SAUCER,SET OF 3 CAKE TINS PANTRY DESIGN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RED RETROSPOT CHARLOTTE BAG,LUNCH BAG WOODLAND,LUNCH BAG BLACK SKULL.,LUNCH BAG CARS BLUE,PACK OF 72 RETROSPOT CAKE CASES,JUMBO STORAGE BAG SUKI,PAPER CHAIN KIT 50'S CHRISTMAS,VICTORIAN GLASS HANGING T-LIGHT,LUNCH BAG SPACEBOY DESIGN,LUNCH BAG SUKI DESIGN,...,SPOTTY BUNTING,PARTY BUNTING,WHITE HANGING HEART T-LIGHT HOLDER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12783,PARTY BUNTING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12784,JUMBO BAG RED RETROSPOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12785,POSTAGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12786,POSTAGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Desta vez, você pode facilitar o processamento preenchendo os campos vazios com a string `nan`:

In [30]:
base.fillna("nan", inplace=True)

Agora converta o dataframe em uma lista de listas, primeiro chamando o atributo `values`, depois o método `tolist`:

In [31]:
transactions = base.values.tolist()

Por curiosidade, inspecione a transação de índice 300.

In [32]:
transactions[300]

['PARTY BUNTING',
 'REGENCY CAKESTAND 3 TIER',
 'SET OF 4 PANTRY JELLY MOULDS',
 'VICTORIAN GLASS HANGING T-LIGHT',
 'LUNCH BAG RED RETROSPOT',
 'LUNCH BAG CARS BLUE',
 'LUNCH BAG WOODLAND',
 'LUNCH BAG  BLACK SKULL.',
 'LUNCH BAG SPACEBOY DESIGN',
 'LUNCH BAG SUKI DESIGN',
 'LUNCH BAG PINK POLKADOT',
 'WHITE HANGING HEART T-LIGHT HOLDER',
 'PACK OF 72 RETROSPOT CAKE CASES',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan']

Agora invoque o algoritmo `apriori`, informando as transações, `min_support=0.02`, `min_confidence=0.25` e `min_lift=5`.

In [33]:
rules = apriori(transactions, min_support=0.02, min_confidence=0.25, min_lift=5)

Execute o algoritmo, convertendo as regras para uma lista.

In [34]:
results = list(rules)

Cheque quantas regras foram geradas.

In [35]:
len(results)

78

Utilize o mesmo código de parsing apresentado em aula para organizar os resultados em um DataFrame.

In [36]:
A = []
B = []
support = []
confidence = []
lift = []

for result in results:
    s = result[1]
    result_rules = result[2]
    for result_rule in result_rules:
        a = list(result_rule[0])
        b = list(result_rule[1])
        c = result_rule[2]
        l = result_rule[3]
        A.append(a)
        B.append(b)
        support.append(s)
        confidence.append(c)
        lift.append(l)

In [37]:
rules_df = pd.DataFrame({
    "A": A,
    "B": B,
    "support": support,
    "confidence": confidence,
    "lift": lift
})

Exiba o DataFrame com os resultados, ordenado de forma decrescente por valor de lift.

In [38]:
rules_df.sort_values(by="lift", ascending=False)

,A,B,support,confidence,lift
13,"[REGENCY CAKESTAND 3 TIER, ROSES REGENCY TEACU...",[GREEN REGENCY TEACUP AND SAUCER],0.029715,0.777096,10.192313
10,[GREEN REGENCY TEACUP AND SAUCER],"[REGENCY CAKESTAND 3 TIER, ROSES REGENCY TEACU...",0.029715,0.389744,10.192313
180,"[nan, REGENCY CAKESTAND 3 TIER, ROSES REGENCY ...",[GREEN REGENCY TEACUP AND SAUCER],0.029637,0.776639,10.186322
173,[GREEN REGENCY TEACUP AND SAUCER],"[nan, REGENCY CAKESTAND 3 TIER, ROSES REGENCY ...",0.029637,0.388718,10.186322
176,"[nan, GREEN REGENCY TEACUP AND SAUCER]","[REGENCY CAKESTAND 3 TIER, ROSES REGENCY TEACU...",0.029637,0.389117,10.175928
...,...,...,...,...,...
193,"[JUMBO BAG RED RETROSPOT, JUMBO BAG PINK POLKA...","[JUMBO SHOPPER VINTAGE RED PAISLEY, nan]",0.029403,0.459096,5.039421
122,"[LUNCH BAG SPACEBOY DESIGN, LUNCH BAG CARS BLUE]",[LUNCH BAG RED RETROSPOT],0.022599,0.604603,5.017298
375,"[nan, LUNCH BAG SPACEBOY DESIGN, LUNCH BAG CAR...",[LUNCH BAG RED RETROSPOT],0.022521,0.603774,5.010420
370,"[LUNCH BAG SPACEBOY DESIGN, LUNCH BAG CARS BLUE]","[nan, LUNCH BAG RED RETROSPOT]",0.022521,0.602510,5.003184
